https://medium.com/swlh/emotion-detection-using-pytorch-4f6fbfd14b2e

In [8]:
from cv2 import cv2
import os
root = os.getcwd()
# Generate image snippets
def snippet(video_clip,root):
    if not os.path.exists("Out"):
        os.mkdir("Out")
    pathOut = root + "\Out\\"

    vid = video_clip
    cap = cv2.VideoCapture(vid)
    count = 0
    # counter += 1
    success = True
    while success:
        success, image = cap.read()
        # print('read a new frame:',success)
        if count % 20 == 0:
            cv2.imwrite(pathOut + 'frame%d.jpg'%count, image)
        count += 1
snippet(root + "\\videos\\happy.mp4", root)

In [ ]:
import json

# packages for audio recog
import speech_recognition as sr
from os import path
from pydub import AudioSegment
from termcolor import colored
import spacy
import re
from afinn import Afinn
import pandas as pd
import moviepy.editor as mp
import os
from pathlib import Path

#########################################################################################################
#                                                                                                       #
#                                           FACIAL RECOG                                                #
#                                                                                                       #
#########################################################################################################
# Video snippet
from cv2 import cv2
import os

def snippet(video_clip):
    if not os.path.exists("Out"):
        os.mkdir("Out")
    pathOut = os.getcwd() + "\Out\\"

    vid = video_clip
    cap = cv2.VideoCapture(vid)
    count = 0
    # counter += 1
    success = True
    while success:
        success, image = cap.read()
        # print('read a new frame:',success)
        if count % 15 == 0:
            cv2.imwrite(pathOut + 'frame%d.jpg'%count, image)
        count += 1

# predicting using model
import numpy as np # linear algebra
import json
import os
import keras
from keras.models import Sequential,model_from_json
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import image

def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(64, (5, 5), input_shape=input_shape, activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
    model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    return model

def getData(filename):
    Y = []
    X = []
    first = True
    for line in open(filename):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y

def facial_main(video_clip):
    snippet(video_clip)
    model=my_model()
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    model = keras.models.load_model('model_1.h5')

    results = []
    directory = os.getcwd() + "\Out"

    snips = os.listdir(directory)
    for pic in snips:
        img = image.load_img(directory + "\\" + pic, grayscale=True, target_size=(48, 48))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        x /= 255

        custom = model.predict(x)
        results.append(custom[0])

    res = [0,0,0,0,0,0,0]
    for i in range(len(results)):
        for j in range(len(results[i])):
            res[j] += results[i][j]

    out = []
    for i in range(len(objects)):
        out.append([objects[i],res[i]])
    return json.dumps(out)
